### The temporal network class

In this notebook we explore how to load the coherence connectivity data with the temporal network class

In [1]:
# Path to the toolbox
import sys

sys.path.insert(1, "/home/vinicius/storage1/projects/GrayData-Analysis")
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

# GDa
from GDa.temporal_network import temporal_network

### Instantiate temporal network

In [5]:
net = temporal_network(
    coh_file="coh_k_0.3_multitaper_at_cue.nc",
    coh_sig_file="coh_k_0.3_multitaper_at_cue_surr.nc",
    date="141017",
    trial_type=[1],
    behavioral_response=[1],
)

In [6]:
net.convert_to_adjacency()

In [11]:
A = net.A.isel(freqs=4, trials=123, times=50)

In [13]:
from GDa.graphics.plot_brain_sketch import plot_edge_brain_sketch

In [16]:
pairs = np.array([net.A.attrs["sources"], net.A.attrs["targets"]]).T

In [18]:
nodes = net.A.attrs["channels_labels"]

In [25]:
adj = A.to_dataframe(name="adj").reset_index()

In [27]:
del adj["trials"], adj["times"], adj["freqs"]

In [29]:
adj.to_csv("adj.csv")